In [46]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from pandas import read_csv
from sklearn.impute import SimpleImputer
import ray 

In [47]:
data=pd.read_csv(r'C:\Users\Sana\Desktop\\Esol.csv')
data.shape

(1128, 201)

In [48]:
data.isnull().sum().sum()

0

In [49]:
Xdata = data.iloc[:,0:200]
Ydata = data.iloc[:,200:201]

In [50]:
Ydata.shape

(1128, 1)

In [51]:
Xdata.shape

(1128, 200)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(Xdata, Ydata, test_size=0.3, random_state=1)

In [53]:
from sklearn import preprocessing

# scaler_data = preprocessing.MinMaxScaler()
# X_train = scaler_data.fit_transform(X_train)
# X_test = scaler_data.transform(X_test)
scaler_labels = preprocessing.MinMaxScaler()
Y_train = scaler_labels.fit_transform(Y_train.values.reshape(-1, 1))
Y_test = scaler_labels.transform(Y_test.values.reshape(-1, 1))

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
#print(X_train)
#print("Train labels before scaling: {} {} {}Train labels after scaling: {} {}".format('\n',train_labels_before,'\n', '\n', train_labels))

(789, 200) (789, 1) (339, 200) (339, 1)


In [54]:
type(X_train)

pandas.core.frame.DataFrame

In [55]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [56]:
# from sklearn.linear_model import LinearRegression
# from sklearn import metrics
# model= LinearRegression(fit_intercept=False)
# model2=model.fit(X_train, Y_train)
# print(model2.summary())

In [57]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=15)
X_train_pca = sklearn_pca.fit_transform(X_train)
print(X_train_pca.shape)

(789, 15)


In [58]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [59]:
# transform to torch tensor

tensor_x = torch.tensor(X_train_pca, dtype=torch.float).to(device) 
tensor_x2 = torch.tensor(X_test, dtype=torch.float).to(device) 

tensor_y = torch.tensor(Y_train, dtype=torch.float).to(device)
tensor_y2 = torch.tensor(Y_test, dtype=torch.float).to(device)

In [60]:
type(tensor_x)

torch.Tensor

In [61]:
trainset = TensorDataset(tensor_x, tensor_y) 
testset = TensorDataset(tensor_x2,tensor_y2)

In [62]:
try:
    import ray
except:
    !conda install -U ray
    import ray

In [63]:
#conda install torchsummary

In [64]:
# pip install torch-summary

In [65]:
from functools import partial
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torchsummary import summary

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [66]:
class Net(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.hidden_dim1 = int(self.config.get("hidden_dim1", 100))
        self.hidden_dim2 = int(self.config.get("hidden_dim2", 100))
        self.hidden_dim3 = int(self.config.get("hidden_dim3", 100))

        self.act1 = self.config.get("act1", "relu") 
        self.act2 = self.config.get("act2", "relu") 
        self.act3 = self.config.get("act3", "relu")

        self.linear1 = nn.Linear(12, self.hidden_dim1)
        self.linear2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.linear3 = nn.Linear(self.hidden_dim2, self.hidden_dim3)
        self.linear4 = nn.Linear(self.hidden_dim3, 1)
    
    @staticmethod
    def activation_func(act_str):
        if act_str=="tanh":
            return eval("torch."+act_str)
        elif act_str=="selu" or act_str=="relu":   
            return eval("torch.nn.functional."+act_str)

    def forward(self, x):
        output = self.linear1(x)
        output = self.activation_func(self.act1)(output)
        output = self.linear2(output)
        output = self.activation_func(self.act2)(output)
        output = self.linear3(output)
        output = self.activation_func(self.act3)(output)
        output = self.linear4(output)        
        return output

In [67]:
model = Net({})

In [69]:
summary(model, (1,tensor_x.shape[1]))

count_parameters(model)

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: []